In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
import numpy as np

In [3]:
tf.set_random_seed(777) # for reproducibility
learning_rate = 0.1

In [4]:
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

In [5]:
x_data = np.array(x_data, dtype = np.float32)
y_data = np.array(y_data, dtype = np.float32)

In [6]:
X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])

In [7]:
W1 = tf.Variable(tf.random_normal([2, 2]), name = 'weight1')
b1 = tf.Variable(tf.random_normal([2]), name = 'bias1')
l1 = tf.sigmoid(tf.matmul(X, W1) + b1)

In [8]:
W2 = tf.Variable(tf.random_normal([2, 1]), name ='weight2')
b2 = tf.Variable(tf.random_normal([1]), name = 'bias2')
Y_pred = tf.sigmoid(tf.matmul(l1, W2) + b2)

In [9]:
# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(Y_pred) + (1 - Y) *
                       tf.log(1 - Y_pred))

In [10]:
# Network
#          p1     a1           l1     p2     a2           l2 (y_pred)
# X -> (*) -> (+) -> (sigmoid) -> (*) -> (+) -> (sigmoid) -> (loss)
#       ^      ^                   ^      ^
#       |      |                   |      |
#       W1     b1                  W2     b2

In [11]:
# Loss derivative
d_Y_pred = (Y_pred - Y) / (Y_pred * (1.0 - Y_pred) + 1e-7)

In [12]:
# Layer 2
d_sigma2 = Y_pred * (1 - Y_pred)
d_a2 = d_Y_pred * d_sigma2
d_p2 = d_a2
d_b2 = d_a2
d_W2 = tf.matmul(tf.transpose(l1), d_p2)

In [13]:
# Mean
d_b2_mean = tf.reduce_mean(d_b2, axis = [0])
d_W2_mean = d_W2 / tf.cast(tf.shape(l1)[0], dtype = tf.float32)

In [14]:
# Layer 1
d_l1 = tf.matmul(d_p2, tf.transpose(W2))
d_sigma1 = l1 * (1 - l1)
d_a1 = d_l1 * d_sigma1
d_b1 = d_a1
d_p1 = d_a1
d_W1 = tf.matmul(tf.transpose(X), d_a1)

In [15]:
# Mean
d_W1_mean = d_W1 / tf.cast(tf.shape(X)[0], dtype = tf.float32)
d_b1_mean = tf.reduce_mean(d_b1, axis = [0])

In [16]:
# Weight update
step = [
        tf.assign(W2, W2 - learning_rate * d_W2_mean),
        tf.assign(b2, b2 - learning_rate * d_b2_mean),
        tf.assign(W1, W1 - learning_rate * d_W1_mean),
        tf.assign(b1, b1 - learning_rate * d_b1_mean)
]

In [17]:
# Accuracy computation
# True if hypothesis > 0.5 else False
predicted = tf.cast(Y_pred > 0.5, dtype = tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32))

In [18]:
# Launch graph
with tf.Session() as sess:
  # Initialize TensorFlow variables
  sess.run(tf.global_variables_initializer())

  print("shape", sess.run(tf.shape(X)[0], feed_dict = {X: x_data}))

  for i in range(10001):
    sess.run([step, cost], feed_dict = {X: x_data, Y: y_data})
    if i % 1000 == 0:
      print(i, sess.run([cost, d_W1],
                        feed_dict = {X: x_data, Y: y_data}), sess.run([W1, W2]))
  
  # Accuracy report
  h, c, a = sess.run([Y_pred, predicted, accuracy],
                     feed_dict = {X: x_data, Y: y_data})
  print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

shape 4
0 [0.7539022, array([[ 0.14797105, -0.05841585],
       [ 0.09894872, -0.06082735]], dtype=float32)] [array([[ 0.7988674 ,  0.6801188 ],
       [-1.2198634 , -0.30361032]], dtype=float32), array([[ 1.3752297 ],
       [-0.78823847]], dtype=float32)]
1000 [0.6712278, array([[-0.05335969, -0.005697  ],
       [ 0.04508774,  0.00137679]], dtype=float32)] [array([[ 1.1574888 ,  0.704702  ],
       [-1.8545096 , -0.15281203]], dtype=float32), array([[ 1.3886381],
       [-0.8744261]], dtype=float32)]
2000 [0.53392583, array([[-0.07984564, -0.06233615],
       [ 0.06752959,  0.05506495]], dtype=float32)] [array([[ 3.0532162,  1.388589 ],
       [-3.4559672, -0.6531812]], dtype=float32), array([[ 3.2835448],
       [-1.5619068]], dtype=float32)]
3000 [0.1978524, array([[-0.04034935, -0.07028399],
       [ 0.0350967 ,  0.08068884]], dtype=float32)] [array([[ 4.623502 ,  3.5030947],
       [-4.7522364, -3.0363069]], dtype=float32), array([[ 5.564976],
       [-4.344845]], dtype=float32)